In [2]:
from pyspark.sql import SparkSession 

#create session
spark = SparkSession.builder.appName("BookRecommendCSV").getOrCreate()

In [3]:
spark


In [4]:
# Read the CSV file with specified options
books = spark.read \
    .format("csv") \
    .option("sep", ";") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .option("mode", "DROPMALFORMED") \
    .load("data/BX-Books.csv")

In [5]:
books.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [8]:
books.columns

['ISBN',
 'Book-Title',
 'Book-Author',
 'Year-Of-Publication',
 'Publisher',
 'Image-URL-S',
 'Image-URL-M',
 'Image-URL-L']

In [11]:
books.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [13]:
rename_dict = {
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"
}

for old_name, new_name in rename_dict.items():
    books = books.withColumnRenamed(old_name, new_name)

In [16]:
books = books.select('ISBN', 'title', 'author', 'year', 'publisher', 'image_url')

In [17]:
books.show()

+----------+--------------------+--------------------+----+--------------------+--------------------+
|      ISBN|               title|              author|year|           publisher|           image_url|
+----------+--------------------+--------------------+----+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux|http://images.ama...|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|W. W. Norton &amp...|http://images.ama...|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group|http://images.ama...|
|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...|ht